# TP3 : Décomposition QR, bataille navale et épidemiologie
### Physique Numérique (PHY-3500)
### Par: Simon Gauthier, Laurent Émond, Thomas Charland
### Présenté à: Xavier Roy-Pomerleau et Antoine Allard
### Remis le: 14 avril 2025

## TP3.1 : Décomposition QR par méthode de Householder

### Mise en contexte

Mise en contexte textuelle ici

### (a) Démontrer que $Q_i$ sont orthogonales

Insérer calculs pour TP3.1.a

### (b) Démontrer équation 3.5 et que matrice $Q$ est orthogonale

Insérer calculs pour TP3.1.b

### (c) Implémenter householder_qr

In [1]:
#Code pour TP3.1.c

Insérer explications textuelles ici

### (d) Comparer avec numpy.linalg.qr

In [2]:
#Code pour TP3.1.d

Insérer explications textuelles ici

### (e) Illustrer la triangularisation de la matrice du numéro précédent

Insérer démonstration Pour TP3.1.e

# TP3.2 : bataille navale

### Mise en contexte

Mise en contexte textuelle ici

### (a) modifier le code de la décomposition QR

In [ ]:
#insérer code pour question TP3.2.a

Insérer explications textuelles ici

### (b) résoudre approximativement l'équation 3.9

In [ ]:
#insérer code pour question TP3.2.b

Insérer explications textuelles ici

### (c) Tracez les données et solutions estimées

In [ ]:
#insérer code pour question TP3.2.c

Insérer explications textuelles ici

### (d) Comparaison de méthodes de résolution

In [ ]:
#insérer code ici

Insérer explications textuelles ici

# TP3.3 : Épidemiologie SIR sur réseau

### Mise en contexte

Mise en contexte textuelle ici

### (a) Solutions analytiques

Solution analytioque en latex ici :)

### (b) Démontrer la solution qui correspond à $u$

Démonstration ici :)

### (c) Tracez $R_{\infty}$ en fonction de $T$

In [ ]:
#insérer code pour TP3.3.c ici

Insérer explications textuelles ici

### (d) Résoudre équation 3.14 numérique, méthode de relaxation et méthode Newton-Raphson

In [ ]:
#insérer code pour TP3.3.d, méthode de relaxation

In [ ]:
#insérer code pour TP3.3.d, méthode de Newton-Raphson

Insérer explications textuelles ici